참고  
https://github.com/fanglanting/skip-gram-pytorch/blob/master/model.py  
https://adoni.github.io/2017/11/08/word2vec-pytorch/  
https://github.com/weberrr/pytorch_word2vec/tree/master/skip_gram_on_hierarchical_softmax   

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import heapq

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)    

cuda


## skip-gram model  
중심 단어로 주변 단어 예측  
일반적으로 CBOW보다 Skip-gram의 성능이 더 좋음.  
**단어별로 인덱스 부여**     

`임베딩 벡터 사용 방법`    
1. 임베딩 층(embedding layer)을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법  
nn.Embedding()로 구현: https://wikidocs.net/64779  
nn.Embedding(num_embeddings. embedding_dim): 학습 가능한 임베딩 테이블을 만듦.    
- num_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기  
- embedding_dim : 임베딩 할 벡터의 차원. 사용자가 정해주는 하이퍼파라미터  

2. 미리 사전에 훈련된 임베딩 벡터들을 가져와 사용하는 방법  
  
밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련됨.  

임베딩 층의 입력이 원-핫 벡터가 아니어도 동작함. 파이토치는 단어를 정수 인덱스로만 바꾼 채로 임베딩 층의 입력으로 사용해도 룩업 테이블 된 결과인 임베딩 벡터를 리턴함.  

http://i.imgur.com/TupGxMl.png


  

In [3]:
# Skip-gram model
class skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(skipgram, self).__init__()
        self.vocab_size = vocab_size  # vocab_size: 임베딩을 할 단어들의 개수(V)
        self.embedding_dim = embedding_dim  # embedding_dim: 임베딩을 할 벡터의 차원(N)
        # nn.Embedding(): 학습 가능한 임베딩 테이블을 만들기 
        self.v_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)  # W(input vector representation of word w)
        self.init_emb()

    # initialize weights: 처음에는 임의로 설정, 학습 진행되며 업데이트 시킬 부분임.
    # self.v_embeddings.weight.data.uniform_: 균일 분포로 초기화시킴.
    def init_emb(self):
        initrange = 0.5 / self.embedding_dim  # 그냥 임의로 설정한 부분
        self.v_embeddings.weight.data.uniform_(-initrange, initrange)

    # forward(순전파): 학습을 하면서 답을 출력하는 과정
    # squeeze: 1인 차원을 제거한다.
    def forward(self, center_id):
        # center_id: center word의 id
        embed_center = self.v_embeddings(torch.LongTensor([center_id])).squeeze()  # embed_center: hidden layer의 벡터

        return embed_center

    # word vector 저장
    def save_embedding(self, file_name, id2word):
        embeds = self.v_embeddings.weight.data.numpy()  # W
        fo = open(file_name, 'w')
        for idx in range(len(embeds)):
            word = id2word[idx]
            embed = ', '.join(map(str, embeds[idx]))  # 공백을 구분자로 word vector 가지고 오기 ex) 0.5 2.3 0.1
            fo.write(word+' '+embed+'\n')

## hierarchical softmax
https://discuss.pytorch.org/t/feedback-on-manually-implemented-hierarchical-softmax/82478   
https://medium.com/syncedreview/word2vec-googles-new-leap-forward-on-the-vectorized-representation-of-words-e8e505a31595  

---

hierarrchical softmax가 필요한 이유: https://www.youtube.com/watch?v=pzyIWCelt_E   
  
**H-Softmax essentially replaces the flat softmax layer with a hierarchical layer that has the words as leaves**  
=> 어차피 softmax 계산 너무 많아서 그 부분을 교체한 것이니까 hierarchical softmax는 확률만 계산하는 부분임. word vector는 skip-gram에서 원래 구해지던 것처럼 구해지겠지!
  
---

**no output word representations**  
negative sampling과 original softmax formulation에서는 input matrix와 output matrix의 모든 row가 특정 word와 대응되었음. 
-> hierarchical softmax에서는 input matrix에서는 마찬가지이지만 output matrix는 그렇지 않음. **output matrix는 internal node로 구성됨((단어개수 - 1)개의 노드)**  

In hierarchical softmax, we have a target word, and a binary tree with each leaf node corresponding to words in the vocabulary.  
  
---

tree: huffman tree https://playground10.tistory.com/98  
아무 tree나 가능하긴 하지만 Huffman Tree를 사용할 경우 자주 등장하는 단어들은 보다 짧은 path로 도달할 수 있음.(root로부터 가까이 위치-얕은 층에) -> 전체적인 계산량이 더 낮아지는 효과를 볼 수 있음.  
또한 Huffman Tree는 Full Binary Tree이기 때문에 Hierarchical Softmax의 조건에도 부합한다.    
https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/ 



### (Huffman)Node 
각 노드마다 vector 가짐. v′n(w,j)  

- 초기화시 좌우 노드는 비어있다.  

https://github.com/weberrr/pytorch_word2vec/blob/master/skip_gram_on_hierarchical_softmax/huffman_tree.py  
https://discuss.pytorch.org/t/feedback-on-manually-implemented-hierarchical-softmax/82478  

In [4]:
class Node:
    def __init__(self, token, freq): 
        self.vec = torch.randn(emb_dimension, requires_grad=True, dtype=torch.float) # 노드마다 벡터 가짐. / 벡터의 사이즈는 emb_dimension
        self.token = token # vocabulary에서 각각의 단어를 의미
        self.freq = freq   # freq: vocabulary에서 단어가 등장하는 횟수
        self.left = None   # left: 왼쪽 노드
        self.right = None  # right: 오른쪽 노드

    # 객체끼리 freq 비교: huffman tree에서는 빈도수가 많은 것이 root에서 얕은 층에 위치하기 때문에 이 과정이 필요함.
    # less than
    def __lt__(self, other): 
        return self.freq < other.freq
    
    # greater than
    def __gt__(self, other): 
        return self.freq > other.freq
    
    # equal
    def __eq__(self, other):
        if (other == None):
            return False
        if (not isinstance(other, Node)): 
        # 인스턴스가 특정 클래스/데이터 타입과 일치하면 True
        # isinstance는 파이썬 자료형을 확인하는 함수임. isinstance(인스턴스, 클래스/데이터타입)
            return False
        return self.freq == other.freq  

### HuffmanTree  
heap: 완전 이진 트리의 일종 / 우선순위 큐를 위하여 만들어진 자료 구조  
  
데이터를 정렬하기 위해서 파이썬 내장 모듈인 heapq 사용함.  
heapq 모듈: 파이썬의 보통 리스트를 마치 최소 힙처럼 다룰 수 있도록 도와줌.  

빈 리스트를 생성해놓은 다음에 heapq 모듈의 함수를 호출할 때마다 이 리스트를 인자로 넘겨야 함.  

최소힙: 부모 노드가 자식 노드보다 크기가 작거나 같도록 원소가 추가되고 삭제됨.  
https://t1.daumcdn.net/cfile/tistory/17084F504DA9895214  
  
---

허프만 코딩: tree의 root에서 예측하고 싶은 단어까지 도달하는 path를 알 수 있음.  
0이 왼쪽, 1이 오른쪽  

---
`make_heap`: 각 단어 노드로 만들고 최소힙으로 정렬 => leaf 노드를 만드는 과정임.  
`merge_nodes`: tree 구조를 만드는 과정임./빈도수가 큰 것을 tree의 root에서 가깝게 만드려는 과정임.(짧은 path로 도달 가능)  
`make_codes_helper`, `make_codes`: (만들어 높은 tree에서) root에서 예측하고 싶은 단어까지 도달하는 path를 알려주는 과정임.  
 



In [5]:
# to construct the binary huffman tree for hierarchical softmax
class HuffmanTree:
    def __init__(self):
        self.heap = [] # 리스트를 최소 힙으로 정렬하려고 함.
        self.codes = {} # 'token : 허프만코드'인 딕셔너리
        self.reverse_mapping = {} # '허프만코드 : token'인 딕셔너리
        self.root = None

    # make_heap: 각 단어 노드로 만들고 최소힙으로 정렬 => leaf 노드를 만드는 과정임.
    # '단어 : 빈도수'로 되어있는 딕셔너리에서 각 단어를 Node 객체로 만들고, 최소힙으로 정렬함.    
    def make_heap(self, frequency): # frequency: token의 frequnecy를 나타내는 딕셔너리
        for key in frequency:       # key는 token, frequnecy[key]는 token의 빈도수
            node = Node(key, frequency[key])
            heapq.heappush(self.heap, node) # heapq.heappush: 힙에 원소 추가 / 자동으로 정렬됨.
            
    # merge_nodes: tree 구조를 만드는 과정/빈도수가 큰 것을 tree의 root에서 가깝게 만드려는 과정임.(짧은 path로 도달 가능)   
    # 빈도수가 작은 것은 미리 합쳐서 merged 노드로 만들고 다른 노드와 비교
    # 원래 vocabulary의 token이 leaf node가 됨.
    # leaf 노드는 token이 있고, merged된 노드는 token이 없음.
    def merge_nodes(self):
        while(len(self.heap)>1):   
            # heappop: 힙에서 원소 삭제 / 가장 작은 원소를 삭제하고 그 값을 리턴함.
            node1 = heapq.heappop(self.heap) # 가장 작은 노드
            node2 = heapq.heappop(self.heap) # 두 번째로 작은 노드
            
            merged = Node(None, node1.freq + node2.freq) # 남은 노드 중 빈도수 가장 작은 두 개의 빈도수 합침.
            merged.left = node1
            merged.right = node2
            heapq.heappush(self.heap, merged) # merged라는 새로운 노드를 힙에 추가
    
    # make_codes_helper, make_codes: (만들어 높은 tree에서) root에서 예측하고 싶은 단어까지 도달하는 path를 알려주는 과정임.

    # make_codes_helper: 각 node에 bit 붙여주는 것을 말함.(예) '01')
    # 여기서 code는 huffman code
    # 허프만 코딩: 입력 파일의 문자 빈도 수를 가지고 최소힙을 이용하여 파일을 압축하는 과정
    def make_codes_helper(self, root, current_code):
        if (root == None):
            return
        if (root.token != None): # root.token != None이라는 말은 merged된 노드가 아니라 leaf 노드라는 뜻.
            self.codes[root.token] = current_code # 'leaf 노드의 token : 허프만 코드'로 codes 딕셔너리에 저장됨.
            self.reverse_mapping[current_code] = root.token # '허프만코드 : leaf 노드의 token'로 reverse_mapping 딕셔너리에 저장됨.
            return
        
        # 왼쪽 노드에는 뒤에 0을, 오른쪽 노드에는 뒤에 1을 붙이기 
        self.make_codes_helper(root.left, current_code + "0")
        self.make_codes_helper(root.right, current_code + "1")
        
    # make_codes: leaf 노드에 허프만 코드(예) '01') 붙여주는 과정
    # make_heap이랑 merge_nodes를 적용한 다음에 make_codes를 쓰면, 
    # self.heap에는 마지막에 merge된 가장 값이 큰 노드밖에 남아있지 않으니까 root는 가장 값이 큰 노드가 됨.
    # make_codes_helper 안에 자기 자신이 들어있으니까 make_codes는 한 번만 써주면 됨.
    def make_codes(self):
        root = heapq.heappop(self.heap) #  make_heap과 merge_nodes를 먼저 적용했으므로 self.heap에 남아 있는 가장 값이 큰 노드가 리턴됨.
        self.root = root # 가장 큰 노드가 root가 됨.
        current_code = "" # 처음에는 아무 것도 없는 데서 시작/root에서 내려갈수록 점점 0(왼쪽), 1(오른쪽) 붙여나갈 것임.
        self.make_codes_helper(root, current_code)

### initialize the leaves of the binary tree as words in vocabulary

In [6]:
# ht corresponds to one instance of a huffman tree for softmax
def initialize_the_leaves(voca_freq_dict):
    ht = HuffmanTree()
    ht.make_heap(voca_freq_dict)
    ht.merge_nodes()
    ht.make_codes()
    return ht

### loss function for hierarchical softmax  
calculation of loss function for hierarchical softmax  

우리는 결국 p(w|w_i)를 최대화, 즉 -log(p(w|w_i))를 최소화하고 싶은 것임.  
그러니까 p(w|w_i) 앞에 log를 붙여서 더하기로 바꾼 다음, 마지막에 - 곱하기  



In [7]:
def cal_loss(h, ht, target): # h: hidden vector(w_i), target: 예측하고자 하는 단어 w 
    path_to_word = ht.codes[target]  # 예측하고자 하는 단어 w의 허프만 코드(0이 왼쪽, 1이 오른쪽으로 가는 것이었음.) 
    loss = torch.zeros(1, requires_grad=True, dtype=torch.float)
    root = ht.root
    for i in path_to_word:
        if (i == '0'):
            loss = loss +  torch.log(torch.sigmoid(torch.dot(root.vec, h)))
            root = root.left # 0이 왼쪽
        else:
            loss = loss +  torch.log(torch.sigmoid(-1*torch.dot(root.vec, h))) #오른쪽이면 
            root = root.right # 1이 오른쪽
    loss = loss*(-1) # 앞에 -를 곱해 loss 식을 -log(p(w|w_i))로 만들어 최소화 문제로 바꾸기
    return loss

## word2vec

input file의 형태:  
skip-gram에는 각각의 단어에 unique index를 부여해주면 될 것 같고,  
hierarchical softmax에서는 '단어:빈도수' 형태의 dictionary가 input으로 들어가면 됨.  
https://github.com/weberrr/pytorch_word2vec/blob/master/skip_gram_on_hierarchical_softmax/input_data.py

### input

In [8]:
from collections import deque

class InputData:
    def __init__(self, input_file_name):
        # input_file_name: corpus 형태
        self.input_file_name = input_file_name
        self.input_file = open(self.input_file_name)
        self.wordId_frequency_dict = dict()  # '단어 id:빈도수' dict
        self.word_freq_dict = dict()  # voca_freq_dict: '단어:빈도수' dict
        self.word_count = 0  # unique한 단어의 개수
        self.word_count_sum = 0  # 단어의 총수(겹치는 것도 모두 count)
        self.sentence_count = 0  # 문장의 개수
        self.id2word_dict = dict()  # '단어 id:단어' dict
        self.word2id_dict = dict()  # '단어:단어 id' dict
        self.word_pairs_queue = deque()  # '중심 단어id:주변 단어id' pairs를 queue에 저장
        self._init_dict()

    def _init_dict(self):
        word_freq = dict() # 단어:단어 빈도수
        for line in self.input_file:
            line = line.strip().split(' ')  
            # .strip(): 양쪽 공백 삭제
            # .split(' '): 공백 기준으로 나눠 리스트 만들기
            self.word_count_sum += len(line) # 겹치는 것 고려하지 않은 단어의 총수
            self.sentence_count += 1  # 문장의 개수
            for word in line:
                try:  # 이미 이전 line에서 등장했던 단어는 단어 빈도수에 1 추가
                    word_freq[word] += 1
                except:  # 이번 line에서 처음 등장하는 단어는 단어 빈도수 1로 설정
                    word_freq[word] = 1

        self.word_freq_dict = word_freq

        word_id = 0  # word_id 0부터 시작해서 unique한 단어에 각각 배정시켜줄 것임.
        for per_word, per_count in word_freq.items():
            self.id2word_dict[word_id] = per_word
            self.word2id_dict[per_word] = word_id
            self.wordId_frequency_dict[word_id] = per_count
            word_id += 1
        self.word_count = len(self.word2id_dict)

    def get_batch_pairs(self, batch_size, window_size):
        while len(self.word_pairs_queue) < batch_size:
            for _ in range(10000):  
                self.input_file = open(self.input_file_name, encoding="utf-8")
                sentence = self.input_file.readline()
                if sentence is None or sentence == '':
                    continue
                wordId_list = []  
                for word in sentence.strip().split(' '):
                    try:
                        word_id = self.word2id_dict[word]
                        wordId_list.append(word_id)
                    except:
                        continue
                for i, wordId_w in enumerate(wordId_list):
                    for j, wordId_v in enumerate(wordId_list[max(i - window_size, 0):i + window_size + 1]):
                        assert wordId_w < self.word_count
                        assert wordId_v < self.word_count
                        if i == j:  
                            continue
                        self.word_pairs_queue.append((wordId_w, wordId_v))
        result_pairs = [] 
        # queue에 저장시킨 training pairs를 result_pairs 안에 저장시킴.
        result_pairs = []  
        for _ in range(batch_size):
            result_pairs.append(self.word_pairs_queue.popleft())
        return result_pairs

    # batch가 몇 개 필요한지를 알기위해 pair의 개수를 세는 과정임.
    def evaluate_pairs_count(self, window_size):
        return self.word_count_sum * (2 * window_size) - (self.sentence_count) * (1 + window_size) * window_size
        # 한 문장에서 문장의 맨 앞과 맨 뒤는 context word가 window_size만큼 나오지 않기 때문에 (1 + window_size) * window_size 만큼 pairs 개수가 줄어듦.

In [9]:
import torch.optim as optim
from tqdm import tqdm

# hyperparameters
emb_dimension = 20
lr = 1e-2
window_size = 5
batch_size = 64

class word2vec:
    def __init__(self, input_file_name, output_file_name):
        self.output_file_name = output_file_name
        self.data = InputData(input_file_name)
        self.model = skipgram(self.data.word_count, emb_dimension)
        self.lr = lr
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr)
        self.ht = initialize_the_leaves(self.data.word_freq_dict)

    def train(self):
        print("SkipGram Training......")
        pairs_count = self.data.evaluate_pairs_count(window_size)  # training pairs의 총 개수
        print("pairs_count", pairs_count)
        batch_count = pairs_count / batch_size  # batch의 총 개수
        print("batch_count", batch_count)
        process_bar = tqdm(range(int(batch_count)))
        for i in process_bar:
            pairs = self.data.get_batch_pairs(batch_size, window_size)  # 한 batch 안에서의 training pairs
            for center_id, context_id in pairs:
                self.optimizer.zero_grad()
                h = self.model.forward(center_id)
                loss = cal_loss(h, self.ht, self.data.id2word_dict[context_id])
                loss.backward()
                self.optimizer.step()

        self.model.save_embedding(self.output_file_name, self.data.id2word_dict)

## test  
실제 vocabulary안의 단어들로 test 해보기


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
if __name__ == '__main__':
    w2v = word2vec(input_file_name='/content/drive/My Drive/오바마 당선 연설 영문.txt', output_file_name="word_embedding.txt")  
    w2v.train()

  0%|          | 0/276 [00:00<?, ?it/s]

SkipGram Training......
pairs_count 17690
batch_count 276.40625


100%|██████████| 276/276 [00:23<00:00, 11.79it/s]


In [12]:
from google.colab import files
files.download("word_embedding.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 검증

In [13]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [16]:
doc1 = np.array([0.015696842, -0.0093580065, 0.0074583353, 0.016942114, -0.009867362, -0.0027436197, 0.005580747, 0.02189526, -0.014639899, 0.0077785105, -0.017226039, 0.015460423, -0.0114577925, 0.01306319, -0.010877952, -0.015626753, -0.011649668, -0.01583607, 0.023777708, 0.024805292])
doc2 = np.array([0.023022784, 0.016514814, 0.010946855, -0.016207227, -0.009865761, 0.016219435, -0.0025317313, 0.022654846, -0.010308436, 0.021124793, 0.019364938, -0.013614759, 0.024545256, -0.008188692, -0.013643605, -0.0025060445, -0.012824315, -0.0037854165, 0.021939605, 0.017108796])

In [17]:
cos_sim(doc1, doc2)

0.2739875901576271

해야할 것  
1. ~~sg model~~  
2. ~~hierarchical softmax 마무리~~  
3. ~~word2vec에 정리하기~~  
4. ~~실제 vocabulary로 test해보기~~  
5. ~~input 파일 만들기(각 단어에 인덱스 부여, 단어 : 빈도수 dict 만들기)~~   
https://discuss.pytorch.org/t/feedback-on-manually-implemented-hierarchical-softmax/82478  
https://github.com/weberrr/pytorch_word2vec/blob/master/skip_gram_on_hierarchical_softmax/huffman_tree.py  
https://github.com/leimao/Two-Layer-Hierarchical-Softmax-PyTorch/blob/master/utils.py#L98  
https://www.daleseo.com/python-heapq/
https://i.stack.imgur.com/2ZxYD.png



